In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from lightgbm import LGBMRegressor
import optuna
from sklearn.base import clone
import pandas as pd
import numpy as np
import joblib
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import learning_curve, KFold  #
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
# Import required libraries
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))
from src.data.data_loader import load_raw_data
from src.features.feature_engineering import FeatureEngineering, FlagClusteringTransformer,PreprocessingFeaturesTransformer, PreprocessingFeatures
from category_encoders import TargetEncoder


In [2]:
raw_data = load_raw_data('../../data/raw/raw_data.csv')

2025-08-12 13:31:36,975 - src.data.data_loader - INFO - Successfully loaded 48665 rows from ../../data/raw/raw_data.csv


In [3]:
feat_to_drop = [
    "cd_type_individual",
    "cd_advertise",
    "cd_client",
    "flg_rain_sensor",
    "flg_diesel",
    "flg_eletrico",
    "flg_benzina",
    "flg_pcd",
    "flg_trade_in",
    "flg_armored",
    "flg_factory_warranty",
    "flg_all_dealership_schedule_vehicle",
    "flg_all_dealership_services",
    "flg_single_owner",
    "priority",
    "cd_model_vehicle",
    "cd_version_vehicle",
    "flg_lincese",
    "flg_tax_paid",
    "n_doors",
    "flg_alloy_wheels",
    "flg_gas_natural",
]

pipeline_pre = Pipeline(
    [
        (
            "preprocessing",
            PreprocessingFeaturesTransformer(
                location_col="city_state",
                fuel_type_column="fuel_type",
                cols_to_drop=feat_to_drop,
                outlier_columns=["vl_advertise", "km_vehicle"],
            ),
        ),
        (
            "feat_engineering",
            FlagClusteringTransformer(
                feature_flag_cols=[
                    "flg_gasolina",
                    "flg_electric_locks",
                    "flg_air_conditioning",
                    "flg_electric_windows",
                    "flg_rear_defogger",
                    "flg_heater",
                    "flg_alarm",
                    "flg_airbag",
                    "flg_abs",
                ]
            ),
        ),
    ]
)

In [4]:
df_processed = pipeline_pre.fit_transform(
            raw_data)

2025-08-12 13:31:36,994 - src.features.feature_engineering - INFO - 📈 Numerical features: 5
2025-08-12 13:31:36,995 - src.features.feature_engineering - INFO - 🏷️ Categorical features: 35
2025-08-12 13:31:36,995 - src.features.feature_engineering - INFO - 🎯 Target variable: ['flg_leads', 'leads']
2025-08-12 13:31:36,996 - src.features.feature_engineering - INFO - Runnig location split...
2025-08-12 13:31:38,585 - src.features.feature_engineering - INFO - Converting flag columns to integer type...
2025-08-12 13:31:38,774 - src.features.feature_engineering - INFO - Flag columns converted to integer type successfully
2025-08-12 13:31:38,775 - src.features.feature_engineering - INFO - Converting fuel type to flag columns...
2025-08-12 13:31:38,777 - src.features.feature_engineering - INFO - Found 7 unique fuel types: ['alcool', 'gasolina', 'gas', 'natural', 'diesel', 'eletrico', 'benzina']
2025-08-12 13:31:38,892 - src.features.feature_engineering - INFO - Removing duplicate columns...
202

In [5]:
df_processed.columns

Index(['leads', 'views', 'phone_clicks', 'cd_vehicle_brand', 'year_model',
       'zip_2dig', 'vl_advertise', 'n_photos', 'km_vehicle', 'vl_market',
       'transmission_type', 'flg_leather_seats', 'flg_parking_sensor', 'city',
       'state', 'flg_alcool', 'flag_cluster'],
      dtype='object')

In [58]:


# Fit the pipeline
X = df_processed.drop('leads', axis=1)  # Features
y = df_processed['leads']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the column names to ensure all required columns exist
print("Columns in X_train:", X_train.columns)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('encoding', ColumnTransformer(
        transformers=[
            ('city', TargetEncoder(cols=['city'], smoothing=0.5, min_samples_leaf=1000), ['city']),
            ('state', TargetEncoder(cols=['state'], smoothing=5, min_samples_leaf=500), ['state']),
            ('scaler', StandardScaler(), ['views', 'phone_clicks', 'vl_advertise', 'km_vehicle', 'vl_market']),
            ('transmission', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ['transmission_type'])
        ],
        remainder='passthrough'  # Keep the other columns unchanged
    )),
    ('model', LGBMRegressor(learning_rate=0.01, reg_alpha=1, num_leaves=20, max_depth=10, n_estimators=300,))  # LightGBM Regressor
])

# Set up K-fold cross-validation with stratification on the training data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Columns in X_train: Index(['views', 'phone_clicks', 'cd_vehicle_brand', 'year_model', 'zip_2dig',
       'vl_advertise', 'n_photos', 'km_vehicle', 'vl_market',
       'transmission_type', 'flg_leather_seats', 'flg_parking_sensor', 'city',
       'state', 'flg_alcool', 'flag_cluster'],
      dtype='object')


In [59]:
    # param = {
    #     "objective": "regression",
    #     "metric": "rmse",
    #     "n_estimators": 1000,
    #     "verbosity": -1,
    #     "bagging_freq": 1,
    #     "early_stopping_rounds":150, 
    #     "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
    #     "num_leaves": trial.suggest_int("num_leaves", 2, 50),
    #     "subsample": trial.suggest_float("subsample", 0.05, 1.0),
    #     "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
    #     "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    #     'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
    # }

In [60]:


prep = clone(pipeline.named_steps["encoding"])

prep.fit(X_train, y_train)
X_tr = prep.transform(X_train)
X_te = prep.transform(X_test)


def objective(trial):
    param = {
        'reg_alpha': 1,
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 150,150)
    }
    model = LGBMRegressor(**param)
    model.fit(X_tr, y_train, eval_set=[(X_te, y_test)])
    preds = model.predict(X_te)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    return rmse
study = optuna.create_study(direction='minimize')
optuna.logging.disable_default_handler()
study.optimize(objective, n_trials=150)


[I 2025-08-12 16:47:21,995] A new study created in memory with name: no-name-d418567a-2bf1-407a-9020-b8080603372e


Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[331]	valid_0's l2: 46.0052
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[188]	valid_0's l2: 47.9835
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[309]	valid_0's l2: 52.246
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[728]	valid_0's l2: 90.1026
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[347]	valid_0's l2: 49.8943
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[833]	valid_0's l2: 52.8825
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[205]	valid_0's l2: 50.9273
Training until validation scores don't improve for 150 rounds
Did not meet early stoppi

Trial 26 failed with parameters: {'n_estimators': 538, 'learning_rate': 0.010450479446745447, 'num_leaves': 43, 'max_depth': 6, 'min_data_in_leaf': 24, 'feature_fraction': 0.8491732747680776, 'early_stopping_rounds': 150} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/algelopelisson/Documents/DataScience/projetos/leads-analysis-prediction/venv/lib/python3.13/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/p8/n44gv6mn7z193xr0hyjsfwgh0000gp/T/ipykernel_23863/463207720.py", line 20, in objective
    model.fit(X_tr, y_train, eval_set=[(X_te, y_test)])
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/algelopelisson/Documents/DataScience/projetos/leads-analysis-prediction/venv/lib/python3.13/site-packages/lightgbm/sklearn.py", line 1398, in fit
    super().fit(
    ~~~~~~~~~~~^
        X,
        ^^
    ...<11 lines>...
        init_model=init

KeyboardInterrupt: 

In [61]:
best_params = study.best_params
best_params["verbosity"] = -1
best_params.pop('early_stopping_rounds', None) 
print(best_params) # Remove early_stopping_rounds if it exists

{'n_estimators': 326, 'learning_rate': 0.019077681491088343, 'num_leaves': 26, 'max_depth': 5, 'min_data_in_leaf': 15, 'feature_fraction': 0.7656754625414294, 'verbosity': -1}


In [62]:
baseline_pred = np.full_like(y_train, y_train.mean(), dtype=float)
baseline_mse = np.mean((y_train - baseline_pred) ** 2)

print("MSE baseline:", baseline_mse)

MSE baseline: 135.5174330920903


## Avaliação de Curva de Aprendizado e Overfitting

Vamos plotar a curva de aprendizado (learning curve) para avaliar se há overfitting/underfitting e também reforçar a análise com o scatter plot de valores reais vs preditos.

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ===== 1) Curva de aprendizado (RMSE) =====
cv = KFold(n_splits=5, shuffle=True, random_state=42)

pipeline.named_steps['model'].set_params(**best_params)

train_sizes, train_scores, test_scores = learning_curve(
    estimator=pipeline,
    X=X_train, y=y_train,
    cv=cv,
    scoring='neg_root_mean_squared_error',  # já é RMSE com sinal negativo
    train_sizes=np.linspace(0.1, 1.0, 8),
    n_jobs=-1,
    shuffle=True,
    random_state=42,
    return_times=False
)

# Como a métrica é "neg_root_mean_squared_error", basta inverter o sinal:
train_rmse = -train_scores
test_rmse  = -test_scores

train_mean = train_rmse.mean(axis=1)
train_std  = train_rmse.std(axis=1)
test_mean  = test_rmse.mean(axis=1)
test_std   = test_rmse.std(axis=1)

gap = test_mean - train_mean
gap_ratio = gap / np.maximum(train_mean, 1e-9)  # proteção contra div/0

# Heurística p/ detectar início de overfitting:
# - limiar relativo (ex.: 10%)
# - precisa manter por algumas frações consecutivas (estabilidade)
threshold = 0.10
min_consecutive = 2
start_idx = None
streak = 0
for i in range(len(train_sizes)):
    if gap_ratio[i] > threshold:
        streak += 1
        if streak >= min_consecutive:
            start_idx = i - min_consecutive + 1
            break
    else:
        streak = 0

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', label='Train RMSE')
plt.plot(train_sizes, test_mean, 'o-', label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15)
plt.fill_between(train_sizes, test_mean - test_std,   test_mean + test_std,   alpha=0.15)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve (RMSE)')
plt.grid(True)
plt.legend()

if start_idx is not None:
    x0 = train_sizes[start_idx]
    y0 = test_mean[start_idx]
    plt.axvline(x0, linestyle='--', alpha=0.6)
    plt.annotate(
        f'Início provável do overfitting\n(gap > {int(threshold*100)}%)',
        xy=(x0, y0),
        xytext=(x0, y0 + (test_mean.max()-test_mean.min())*0.15),
        arrowprops=dict(arrowstyle='->', lw=1),
        ha='center'
    )
plt.show()

# Diagnóstico textual (último ponto)
last_gap = gap[-1]
last_ratio = gap_ratio[-1]
if last_ratio > threshold:
    print(f"⚠️ Possível overfitting (último ponto): gap={last_gap:.3f}, gap/train={last_ratio:.1%} (> {threshold:.0%})")
else:
    print(f"✅ Sem sinais claros no último ponto: gap={last_gap:.3f}, gap/train={last_ratio:.1%} (≤ {threshold:.0%})")

# ===== 2) Fit final no treino e avaliação no teste =====
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_test, y_pred)
print(f"Teste: RMSE={rmse:.3f} | MSE={mse:.3f} | R²={r2:.3f}")

# ===== 3) Scatter Real vs Predito + linha ideal =====
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, s=20, edgecolor=None)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
plt.plot(lims, lims, 'r--', label='Ideal')
plt.xlabel('Valores Reais')
plt.ylabel('Valores Preditos')
plt.title('Valores Reais vs. Preditos')
plt.legend()
plt.grid(True)
plt.show()

# (Opcional) Resíduos
res = y_test - y_pred
plt.figure(figsize=(8, 4))
sns.histplot(res, bins=40, kde=True)
plt.title('Distribuição dos Resíduos (y_true - y_pred)')
plt.xlabel('Resíduo')
plt.grid(True)
plt.show()
